In [16]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import panel as pn
from panel.interact import interact
from panel import widgets
pn.extension('plotly')
import hvplot.pandas
from pathlib import Path
import datetime as dt 
import alpaca_trade_api as tradeapi
from datetime import datetime
from dotenv import load_dotenv

In [17]:
load_dotenv()

True

In [18]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("APCA_API_KEY")
alpaca_secret_key = os.getenv("APCA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [19]:
type(alpaca_api_key)

str

In [23]:
# Format current date as ISO format
start_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()

end_date = pd.Timestamp("2020-08-10", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["TSLA", "DAL", "DIS", "AAL", "MSFT", "AAPL", "GE", "F"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
shares_df = api.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

close_price= shares_df.xs("close", axis='columns', level=1)


In [3]:
# Import data
tickers_users_path= Path("../data/cleandata/Processed_Data/tickers_users.csv")
corr_daily_return_prices_users_path= Path("../data/cleandata/Processed_Data/correl_daily_ret_prices_users.csv")
corr_prices_users_path= Path("../data/cleandata/Processed_Data/correl_prices_users.csv")
daily_return_prices_users_path= Path("../data/cleandata/Processed_Data/daily_ret_prices_users.csv")
df_popularity_path= Path("../data/cleandata/Processed_Data/df_popularity.csv")
prices_users_path= Path("../data/cleandata/Processed_Data/prices_users.csv")
tickers_price_path= Path("../data/cleandata/Processed_Data/tickers_prices.csv")
# Read data
tickers_users= pd.read_csv(tickers_users_path, infer_datetime_format=True, parse_dates=True)
tickers_users= tickers_users.dropna()

corr_daily_return_prices_users = pd.read_csv(corr_daily_return_prices_users_path, infer_datetime_format=True, parse_dates=True)

corr_prices_users = pd.read_csv(corr_prices_users_path, infer_datetime_format=True, parse_dates=True)

daily_return_prices_users = pd.read_csv(daily_return_prices_users_path, infer_datetime_format=True, parse_dates=True)

df_popularity = pd.read_csv(df_popularity_path, infer_datetime_format=True, parse_dates=True)

prices_users = pd.read_csv(prices_users_path, infer_datetime_format=True, parse_dates=True)

tickers_price= pd.read_csv(tickers_price_path, infer_datetime_format=True, parse_dates=True)


In [46]:
# Define Panel Visualization Functions
def robintrack_ABB_stock_price ():
    """Robintrack users accounts effect on ABB's Stock price."""
    abb_robintrack_stock_price = daily_return_prices_users.loc[:,['ABB', 'U_ABB']]
    
    abb_corr_plot = abb_robintrack_stock_price.hvplot.line(title='RobinTrack ABB Accounts Effect in ABB Stock Price')
    return abb_corr_plot
    
def robintrack_TSLA_stock_price ():
    """Robintrack AAB's users accounts effect on TSLA's Stock price."""
    tsla_robintrack_stock_price = daily_return_prices_users.loc[:,['TSLA', 'U_TSLA']]
    
    tsla_corr_plot = tsla_robintrack_stock_price.hvplot.line(title='RobinTrack Users Accounts Effect in TSLA Stock Price')
    return tsla_corr_plot

def robintrack_AMD_stock_price ():
    """Robintrack AMD's users accounts effect on AMD's Stock price."""
    amd_robintrack_stock_price = daily_return_prices_users.loc[:,['AMD', 'U_AMD']]
    
    amd_corr_plot = amd_robintrack_stock_price.hvplot.line(title='RobinTrack Users Accounts Effect in AMD Stock Price')
    return amd_corr_plot

def robintrack_RIOT_stock_price ():
    """Robintrack RIOT's users accounts effect on RIOT's Stock price."""
    riot_robintrack_stock_price = daily_return_prices_users.loc[:,['RIOT', 'U_RIOT']]
    
    riot_corr_plot = riot_robintrack_stock_price.hvplot.line(title='RobinTrack Users Accounts Effect in AMD Stock Price')
    return riot_corr_plot
    
def volatility_moving_average_stocks ():
    """Rolling volatility moving average 21 day for ABB, TSLA, AMD, and RIOT."""
    stocks_daily_returns = daily_return_prices_users.loc[:,['ABB','TSLA', 'AMD', 'RIOT']].dropna()
    ma_stocks_21 = stocks_daily_returns.rolling(21).std().shift()
    plot_ma= ma_stocks_21.hvplot(title= 'Moving Average 21 days', legend=True, figsize=(15,10))
    return plot_ma

def robintrack_popularity_stocks ():
    """RobinTrack popularity impact on stocks."""
    plot_popularity = df_popularity.hvplot.bar(title=" Robintrack Highest popularity", x='ticker', y='users')
    plot_stocks = close_price.hvplot.line(title="Stocks most impacted by Robintrack popularity")
    return plot_popularity + plot_stocks

# Panel Dashboard

In [49]:
corr__users_prices= pn.Column("## RobinTrack user accounts effect on ABB Stock Price", robintrack_ABB_stock_price (), 
                    "## RobinTrack user accounts effect on TSLA Stock Price", robintrack_TSLA_stock_price (),
                             "## RobinTrack user accounts effect on AMD Stock Price", robintrack_AMD_stock_price (),
                             "## RobinTrack user accounts effect on RIOT Stock Price", robintrack_RIOT_stock_price ())

volatility_ma = pn.Row("##Rolling volatility moving average 21 day for ABB, TSLA, AMD, and RIOT", volatility_moving_average_stocks () )

popularity_stock = pn.Column("##Robintrack Highest popularity.", robintrack_popularity_stocks () )

# Create tabs

robintrack_stock_analysis_dashboard = pn.Tabs(("Correlation Analysis", corr__users_prices), ("Volatility", volatility_ma),
                                              ("RobinTrack Popularity", popularity_stock ))

dashboard= pn.Column(pn.Row("Effect of Number of Individual Robinhood on Unerlying Price & Option Volatility for RIOT, AMD, ABB, and TSLA"), 
                     robintrack_stock_analysis_dashboard)

# Serve the Panel Dashboard

In [50]:
dashboard.servable()

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] HoloViews(NdOverlay)
            [2] Markdown(str)
            [3] HoloViews(NdOverlay)
            [4] Markdown(str)
            [5] HoloViews(NdOverlay)
            [6] Markdown(str)
            [7] HoloViews(NdOverlay)
        [1] Row
            [0] Markdown(str)
            [1] HoloViews(NdOverlay)
        [2] Column
            [0] Markdown(str)
            [1] HoloViews(Layout)